<a id = 'top'></a>
Contents:

<a href='#1'>Prediction Models for Acquirer</a> <br>
<a href='#2'>Prediction Models for Acquired</a> <br>

#### Document Processing

In [17]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
import nltk
import re
from contextlib import redirect_stdout
from io import StringIO
import random
from nltk.corpus import stopwords

#Data cleaning
from sklearn.feature_extraction.text import CountVectorizer

#loading and Saving
import joblib

In [13]:
def clean_text_pharmalist(text):   
    # remove '\n'
    text = re.sub("\n", " ", text) 
    # lowercase text
    text = text.lower()
    # remove special characters and replace with space
    text = re.sub('[^0-9a-z\-\&]', ' ', text) 
    # remove extra spaces
    text = [text for text in text.split()]
    
    return text

In [14]:
companies = pd.read_csv(r'C:\Users\User\Desktop\GMGC Research\Wyeth-Pfizer\peercomparison.csv', encoding = 'utf-8')
companies = companies['Company Name']
companies = pd.Series.tolist(companies)
companies = ' '.join(companies)
companies = clean_text_pharmalist(companies)

newlist = []
for i in companies:
    if i not in newlist:
        newlist.append(i)

companies = newlist

In [18]:
stop_words_smodel = stopwords.words('english')

for i in 'abcdefghijklmnopqrstuvwxyz':
    stop_words_smodel.append(i)
stop_words_smodel.extend(companies) 
stop_words_smodel.extend(['ii','inc','usa', 'llc', 'iii', 'ltd', 'corp', 'corporation','co', 'jr', 'wsj' ]) 
stop_words_smodel = list(stop_words_smodel)

In [15]:
def clean_text_smodel(text):   
    # remove '\n' and lower case
    text = re.sub("\n", ' ', text) 
    text = text.lower()
    #remove extra spaces
    # remove special characters and replace with space
    text = re.sub('[^a-z\-\&]', ' ', text) 
    # remove extra spaces
    text = [text for text in text.split()]
    # remove stopwords (which includes single alphabeters)
    text = ' '.join([w for w in text if not w in stop_words_smodel])
    return text

In [24]:
def doc_processor(file_dir):
    
    file = open(file_dir, encoding = 'utf-8', mode = 'r+', newline = '\n')
    text = file.read()
    text = nltk.sent_tokenize(text)
   
    a = []
    
    for i in range(len(text)):
         a.append(clean_text_smodel(text[i]))
    text_final = a
    
    try:
        text_final.remove('')
    
    except Exception:
        pass
    
    return text_final
    
    

In [26]:
test = doc_processor(r'C:\Users\User\Desktop\GMGC Research\Wyeth-Pfizer\Latest\Test News\BMY Announcement.txt')

In [85]:
test

['shareholders approve acquisition company nyse bmy today announced shareholders voted approve issuance shares common stock connection company pending merger nasdaq celg special meeting stockholders',
 'pleased outcome today special meeting thank shareholders support combination said giovanni caforio chairman chief executive officer',
 'together create premier innovative biopharma company leading scientific capabilities well positioned address needs patients high-value innovative',
 'look forward bringing companies together believe deliver significant shareholder value shares voted special meeting voted favor merger agreement',
 'file final vote results certified independent inspector election form -k securities exchange commission',
 'transaction remains track close third quarter subject satisfaction customary closing conditions regulatory approvals']

<a id = '1'></a>
### Prediction models for Acquirer

<a href='#top'>To Top</a> <br>


#### Loading Files

Acquirer

In [104]:
import joblib
svc_clf_aqr = joblib.load(r'C:\Users\User\Desktop\GMGC Research\Sentiment Prototype v1.0\Pickled files\Acquirer\svc_gs_clf_aqr.pkl')

In [105]:
import joblib
cnb_clf_aqr = joblib.load(r'C:\Users\User\Desktop\GMGC Research\Sentiment Prototype v1.0\Pickled files\Acquirer\cnb_gs_clf_aqr.pkl')


In [106]:
import joblib
sgd_clf_aqr = joblib.load(r'C:\Users\User\Desktop\GMGC Research\Sentiment Prototype v1.0\Pickled files\Acquirer\sgd_gs_clf_aqr.pkl')


#### Support Vector Classifier

In [107]:
def svc_predictor_aqr(test):

    array = svc_clf_aqr.predict(test)
    binary_array = []
    for i in array:
        if i == 'sell':
            binary_array.append(-1)
        else:
            binary_array.append(1)
    
    sentiment = sum(binary_array)/len(array)
    percentage = abs(sentiment) * 100
    
    if sentiment < 0:
        print('Overall News Sentiment: Sell')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    elif sentiment > 0:
        print('Overall News Sentiment: Buy')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    else:
        print('This News article is Neutral or undetermined by this model')
        
    svc_parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),  # unigrams or bigrams or trigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'svc_clf__tol': (1e-3, 1e-4, 1e-5),
    'svc_clf__C': (5,6,7),
    'svc_clf__penalty': ('l2', 'l1')
    }
    print("Best parameters set:")
    
    svc_best_parameters = svc_clf_aqr.best_estimator_.get_params()
    for param_name in sorted(svc_parameters.keys()):
        print("\t%s: %r" % (param_name, svc_best_parameters[param_name]))
    
    
    return 
            

In [108]:
svc_predictor_aqr(['announced merger'])

Overall News Sentiment: Sell
Percentage of News that supports sentiment: 100.0 %
Best parameters set:
	svc_clf__C: 5
	svc_clf__penalty: 'l2'
	svc_clf__tol: 0.0001
	tfidf__norm: 'l2'
	tfidf__use_idf: False
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (1, 2)


#### Compliment Naive Bayes

In [89]:
def cnb_predictor_aqr(test):

    array = cnb_clf_aqr.predict(test)
    binary_array = []
    for i in array:
        if i == 'sell':
            binary_array.append(-1)
        else:
            binary_array.append(1)
    
    sentiment = sum(binary_array)/len(array)
    percentage = abs(sentiment) * 100
    
    if sentiment < 0:
        print('Overall News Sentiment: Sell')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    elif sentiment > 0:
        print('Overall News Sentiment: Buy')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    else:
        print('This News article is Neutral or undetermined by this model')
        
    array_proba = cnb_clf.predict_proba(test)
        
    cnb_parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),  # unigrams or bigrams or trigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'cnb_clf__alpha': (0.4, 0.3, 0.5, 0.6),
    'cnb_clf__norm': (True,False),
    }
    print("Best parameters set:")
    
    cnb_best_parameters = cnb_clf_aqr.best_estimator_.get_params()
    for param_name in sorted(cnb_parameters.keys()):
        print("\t%s: %r" % (param_name, cnb_best_parameters[param_name]))
    
    
    return 
            

In [101]:
cnb_predictor_aqr(test)

Overall News Sentiment: Sell
Percentage of News that supports sentiment: 100.0 %
Best parameters set:
	cnb_clf__alpha: 0.5
	cnb_clf__norm: True
	tfidf__norm: 'l2'
	tfidf__use_idf: True
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (1, 3)


#### Stochastic Gradient Descent

In [122]:
def sgd_predictor_aqr(test):

    array = sgd_clf_aqr.predict(test)
    binary_array = []
    for i in array:
        if i == 'sell':
            binary_array.append(-1)
        else:
            binary_array.append(1)
    
    sentiment = sum(binary_array)/len(array)
    percentage = abs(sentiment) * 100
    
    if sentiment < 0:
        print('Overall News Sentiment: Sell')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    elif sentiment > 0:
        print('Overall News Sentiment: Buy')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    else:
        print('This News article is Neutral or undetermined by this model')
        
    array_proba = cnb_clf.predict_proba(test)
        
    sgd_parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),  # unigrams or bigrams or trigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'sgd_clf__alpha': (0.00001, 0.000001),
    'sgd_clf__penalty': ('l2', 'elasticnet'),
    }
    print("Best parameters set:")
    
    sgd_best_parameters = sgd_clf_aqr.best_estimator_.get_params()
    for param_name in sorted(sgd_parameters.keys()):
        print("\t%s: %r" % (param_name, sgd_best_parameters[param_name]))
    
    
    return 
            

In [123]:
sgd_predictor_aqr(test)

Overall News Sentiment: Sell
Percentage of News that supports sentiment: 100.0 %
Best parameters set:
	sgd_clf__alpha: 1e-05
	sgd_clf__penalty: 'elasticnet'
	tfidf__norm: 'l2'
	tfidf__use_idf: True
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (1, 3)


<a id = '2'></a>
### Prediction models for Acquired

<a href='#top'>To Top</a> <br>


Acquired

In [126]:
import joblib
svc_clf_aqd = joblib.load(r'C:\Users\User\Desktop\GMGC Research\Sentiment Prototype v1.0\Pickled files\Acquired\svc_gs_clf_aqd.pkl')

In [127]:
import joblib
sgd_clf_aqd = joblib.load(r'C:\Users\User\Desktop\GMGC Research\Sentiment Prototype v1.0\Pickled files\Acquired\sgd_gs_clf_aqd.pkl')

In [128]:
import joblib
cnb_clf_aqd = joblib.load(r'C:\Users\User\Desktop\GMGC Research\Sentiment Prototype v1.0\Pickled files\Acquired\cnb_gs_clf_aqd.pkl')

#### Support Vector Classifer

In [139]:
def svc_predictor_aqd(test):

    array = svc_clf_aqd.predict(test)
    binary_array = []
    for i in array:
        if i == 'sell':
            binary_array.append(-1)
        else:
            binary_array.append(1)
    
    sentiment = sum(binary_array)/len(array)
    percentage = abs(sentiment) * 100
    
    if sentiment < 0:
        print('Overall News Sentiment: Sell')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    elif sentiment > 0:
        print('Overall News Sentiment: Buy')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    else:
        print('This News article is Neutral or undetermined by this model')
        
    svc_parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),  # unigrams or bigrams or trigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'svc_clf__tol': (1e-3, 1e-4, 1e-5),
    'svc_clf__C': (5,6,7),
    'svc_clf__penalty': ('l2', 'l1')
    }
    print("Best parameters set:")
    
    svc_best_parameters = svc_clf_aqd.best_estimator_.get_params()
    for param_name in sorted(svc_parameters.keys()):
        print("\t%s: %r" % (param_name, svc_best_parameters[param_name]))
    
    
    return             

In [140]:
svc_predictor_aqd(test)

Overall News Sentiment: Buy
Percentage of News that supports sentiment: 100.0 %
Best parameters set:
	svc_clf__C: 5
	svc_clf__penalty: 'l2'
	svc_clf__tol: 0.0001
	tfidf__norm: 'l1'
	tfidf__use_idf: True
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (1, 3)


#### Complement Naive Bayesian

In [141]:
def cnb_predictor_aqd(test):

    array = cnb_clf_aqd.predict(test)
    binary_array = []
    for i in array:
        if i == 'sell':
            binary_array.append(-1)
        else:
            binary_array.append(1)
    
    sentiment = sum(binary_array)/len(array)
    percentage = abs(sentiment) * 100
    
    if sentiment < 0:
        print('Overall News Sentiment: Sell')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    elif sentiment > 0:
        print('Overall News Sentiment: Buy')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    else:
        print('This News article is Neutral or undetermined by this model')
        
    cnb_parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),  # unigrams or bigrams or trigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'cnb_clf__alpha': (0.4, 0.3, 0.5, 0.6),
    'cnb_clf__norm': (True,False),
    }
    
    print("Best parameters set:")
    
    cnb_best_parameters = cnb_clf_aqd.best_estimator_.get_params()
    for param_name in sorted(cnb_parameters.keys()):
        print("\t%s: %r" % (param_name, cnb_best_parameters[param_name]))
    
    
    return             

In [142]:
cnb_predictor_aqd(test)

Overall News Sentiment: Buy
Percentage of News that supports sentiment: 100.0 %
Best parameters set:
	cnb_clf__alpha: 0.5
	cnb_clf__norm: True
	tfidf__norm: 'l1'
	tfidf__use_idf: False
	vect__max_df: 0.5
	vect__max_features: 50000
	vect__ngram_range: (1, 3)


#### Stochastic Gradient Descent

In [143]:
def sgd_predictor_aqd(test):

    array = sgd_clf_aqd.predict(test)
    binary_array = []
    for i in array:
        if i == 'sell':
            binary_array.append(-1)
        else:
            binary_array.append(1)
    
    sentiment = sum(binary_array)/len(array)
    percentage = abs(sentiment) * 100
    
    if sentiment < 0:
        print('Overall News Sentiment: Sell')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    elif sentiment > 0:
        print('Overall News Sentiment: Buy')
        print('Percentage of News that supports sentiment:', percentage, '%')
        
    else:
        print('This News article is Neutral or undetermined by this model')
        
    sgd_parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),  # unigrams or bigrams or trigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'sgd_clf__alpha': (0.00001, 0.000001),
    'sgd_clf__penalty': ('l2', 'elasticnet'),
    }
    
    print("Best parameters set:")
    
    sgd_best_parameters = sgd_clf_aqd.best_estimator_.get_params()
    for param_name in sorted(sgd_parameters.keys()):
        print("\t%s: %r" % (param_name, sgd_best_parameters[param_name]))
    
    
    return             

In [144]:
sgd_predictor_aqd(test)

Overall News Sentiment: Buy
Percentage of News that supports sentiment: 66.66666666666666 %
Best parameters set:
	sgd_clf__alpha: 1e-05
	sgd_clf__penalty: 'elasticnet'
	tfidf__norm: 'l2'
	tfidf__use_idf: True
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (1, 3)
